# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.130250e+02     1.550134e+00
 * time: 0.3765408992767334
     1     1.053623e+01     9.915725e-01
 * time: 2.456576108932495
     2    -1.221911e+01     1.048761e+00
 * time: 3.139741897583008
     3    -3.435428e+01     8.119200e-01
 * time: 4.151794910430908
     4    -4.807960e+01     5.503237e-01
 * time: 5.171741962432861
     5    -5.717580e+01     1.656396e-01
 * time: 6.1815149784088135
     6    -5.994514e+01     8.371992e-02
 * time: 6.848211050033569
     7    -6.090048e+01     5.450259e-02
 * time: 7.5334179401397705
     8    -6.128442e+01     6.251825e-02
 * time: 8.231868028640747
     9    -6.160231e+01     3.291677e-02
 * time: 8.909954071044922
    10    -6.182472e+01     2.927843e-02
 * time: 9.587371110916138
    11    -6.198032e+01     2.215352e-02
 * time: 10.258449077606201
    12    -6.205133e+01     1.778205e-02
 * time: 10.93670105934143
    13    -6.210711e+01     1.543243e-02
 * time: 11.613270998001099
  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671068
    AtomicLocal         -18.8557681
    AtomicNonlocal      14.8522646
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485375 
    Xc                  -19.3336822

    total               -62.261666462002
